In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

title = 'Product Manager'
location = 'New York'

job_list = []


In [2]:
for page_start in range(0, 1000, 100): 
    list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Product%2BManager&location=New%20York%2C%20NY&start={page_start}'
    response = requests.get(list_url)
    list_soup = BeautifulSoup(response.text, "html.parser")
    page_jobs = list_soup.find_all("li")

    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]

        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        job_response = requests.get(job_url)
        job_soup = BeautifulSoup(job_response.text, "html.parser")
        job_post ={}

        try:
            job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            job_post["company_name"] = None
        try:
            job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
        except:
            job_post["job_title"] = None
        try:
            job_post["posted_time"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job_post["posted_time"] = None
        try:
            job_post["num_applicants"] = job_soup.find("figcaption", {"class": "num-applicants__caption"}).text.strip()
        except:
            job_post["num_applicants"] = None

        job_list.append(job_post)


In [3]:
custom_jobs_df = pd.DataFrame(job_list)
custom_jobs_df.to_csv('custom_jobs_data.csv', index=False)


In [4]:
jobs_df = pd.DataFrame(job_list)
jobs_df.to_csv('PM_NY.csv', index=False)